<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://cloud-training-demos-ml/...
ServiceException: 409 Bucket cloud-training-demos-ml already exists.


<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,5896567601480310696,342825
1,9068386407968572094,330863
2,2700219941840496344,307395
3,260598435387740869,320332
4,454960867574323744,346515


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13340 examples in the train dataset and 3394 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.625790,True,24,1,30.0,774501970389208065
1,8.062305,True,35,1,37.0,774501970389208065
2,4.750962,True,22,1,37.0,774501970389208065
3,6.250105,True,31,1,38.0,774501970389208065
4,8.311427,False,32,1,40.0,774501970389208065


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13327.000000,13340.000000,13340.000000,13259.000000,1.334000e+04
mean,7.222158,27.390705,1.036282,38.602459,4.391328e+18
std,1.323833,6.192780,0.196002,2.616937,2.798814e+18
min,0.500449,14.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.035849,32.000000,1.000000,40.000000,7.108882e+18
max,12.187154,50.000000,4.000000,47.000000,9.183606e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.625790,True,24,Single(1),30.0,774501970389208065
1,8.062305,True,35,Single(1),37.0,774501970389208065
2,4.750962,True,22,Single(1),37.0,774501970389208065
3,6.250105,True,31,Single(1),38.0,774501970389208065
4,8.311427,False,32,Single(1),40.0,774501970389208065


In [17]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13335,6.018620,Unknown,35,Single(1),34.0,6637442812569910270
13336,9.565858,Unknown,24,Single(1),39.0,6637442812569910270
13337,8.187968,Unknown,32,Single(1),39.0,6637442812569910270
13338,6.750554,Unknown,22,Single(1),38.0,6637442812569910270
13339,6.375769,Unknown,30,Single(1),37.0,6637442812569910270


In [18]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26502.000000,26502.000000,26502.000000,2.650200e+04
mean,7.223034,27.387292,38.608784,4.391982e+18
std,1.322604,6.189689,2.593818,2.796488e+18
min,0.500449,14.000000,17.000000,1.244589e+17
25%,6.563162,22.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.042463,32.000000,40.000000,7.108882e+18
max,12.187154,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [19]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [20]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6726 eval.csv
  26502 train.csv
  33228 total
==> eval.csv <==
6.9996768185,False,20,Single(1),37.0,6392072535155213407
5.03976730932,True,33,Single(1),37.0,1088037545023002395
8.93754010148,True,37,Single(1),43.0,6392072535155213407
6.0186197526,True,33,Single(1),40.0,6141045177192779423
8.3224503905,True,36,Single(1),40.0,1569531340167098963
7.25100379718,False,18,Single(1),32.0,6910174677251748583
8.7633749145,False,24,Single(1),43.0,6392072535155213407
7.44280596512,True,28,Single(1),39.0,6782146986770280327
6.77260068864,False,31,Single(1),38.0,6141045177192779423
7.5618555866,True,33,Single(1),39.0,6141045177192779423

==> train.csv <==
7.62578964258,True,24,Single(1),30.0,774501970389208065
8.062304921339999,True,35,Single(1),37.0,774501970389208065
4.7509617461,True,22,Single(1),37.0,774501970389208065
6.2501051276999995,True,31,Single(1),38.0,774501970389208065
8.3114272774,False,32,Single(1),40.0,774501970389208065
8.68841774542,False,21,Single(1),40.0,774501970389208065
7

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License